In [ ]:
import json
import time

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import pandas as pd
from gurobipy import GRB
from tqdm.auto import tqdm

from orc.branch import (
    branch_beasley, branch_cost,
    branch_reduced_costs
)
from orc.callbacks import (
    ColumnInclusionCallback,
    LagrPenaltiesReductionCallback,
    OptReductionCallback,
    PrimalHeurCallback
)
from orc.data_structures import (
    BranchAndBound, TimeLimitException
)
from orc.primal import dobson, greedy, hall_hochbaum
from orc.relaxation import dual_lb, lp_rel, subgrad_opt
from orc.utils import generate_problem

mpl.rcParams["font.family"] = ["serif"]
mpl.rcParams["font.serif"] = ["cmr10"]
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

# Primal heuristics comparisons

## Without fixed variables

In [ ]:
table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Columns": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
with open("results/primal.json", "w") as f:
    out = {"output": table}
    json.dump(out, f)

In [ ]:
with open("results/primal.json", "r") as f:
    table = json.load(f)["output"]

In [ ]:
df = pd.DataFrame(table)
df = df.set_index(["Rows", "Columns"])
df

In [ ]:
s = df.style.highlight_max(axis=1, props="bfseries: ;")
print(s.to_latex(column_format="llccc", 
                 hrules=True, 
                 position_float="centering"))

## With fixed variables

In [ ]:
np.random.seed(42)

table = []
funcs = [(greedy, "Greedy"), (dobson, "Dobson"),
         (hall_hochbaum, "Hall-Hochbaum")]
for rows, cols in tqdm(
    [(5, 10), (10, 20), (20, 50), (50, 100)], leave=False):
    res = {"Rows": rows, "Cols": cols, "Greedy": 0, 
           "Dobson": 0, "Hall-Hochbaum": 0}
    for i in tqdm(range(10)):
        A, b = generate_problem(rows, cols)
        fixed_n = int(cols * 0.3) 
        fixed = np.random.choice(
            A.shape[-1], fixed_n, replace=False)
        x0 = fixed[:int(fixed_n / 2)]
        x1 = fixed[int(fixed_n / 2) + 1:]
        x = np.ones(A.shape[-1])
        x[x0] = 0
        if np.any(A @ x < b):
            continue
        v = []
        for f, name in funcs:
            ub = np.sum(A, axis=0) @ f(A, b, [], [])
            v.append(ub)
        j = np.argmin(v)
        best = funcs[j][1]
        res[best] += 1
    table.append(res)

In [ ]:
table

# Subgradient optimization lower bounds

In [ ]:
data = {}
omegas = [10, 20, 40, 50, 70, 100, 150, 200, 300]
for rows, cols in tqdm(
    [(10, 20), (20, 30), (40, 50), (50, 60)], leave=False):
    res = []
    for _ in range(10):
        A, b = generate_problem(rows, cols)
        x = hall_hochbaum(A, b, [], [])
        ub = np.sum(A, axis=0) @ x

        prob_res = []
        for omega in omegas:
            lb = subgrad_opt(A, b, ub, [], [], omega=omega)
            prob_res.append(lb)

        prob_res = np.array(prob_res)
        if prob_res.max() == prob_res.min():
            prob_res = np.ones_like(prob_res)
        else:
            prob_res = (prob_res - prob_res.min()) / \
                (prob_res.max() - prob_res.min())
        res.append(prob_res)
    
    data[(rows, cols)] = list(zip(omegas,np.array(res).mean(axis=0)))

In [ ]:
fig, ax = plt.subplots()
for k, v in data.items():
    x = np.array(v)[:,0]
    y = np.array(v)[:,1]
    ax.plot(x, y, label=f"{k[0]} rows, {k[1]} columns")
ax.legend()
ax.set_xlabel("Iterations")
ax.set_ylabel("Scaled lower bound")
ax.set_title("Subgradient optimization lower bounds")
plt.savefig("report/img/subgrad_lb.png", 
            dpi=300, bbox_inches="tight")

In [ ]:
gap_ls = []
for _, v in data.items():
    gap_prob = []
    for omega, val in v:
        gap_prob.append(val)
    gap_ls.append(gap_prob)

# omega = 150 at index 6
np.max(1 - np.array(gap_ls)[:,6]).round(3)

# Branch and Bound models

In [ ]:
lagr_callback = LagrPenaltiesReductionCallback()
col_callback = ColumnInclusionCallback()
opt_red_callback = OptReductionCallback()
primal_heur = PrimalHeurCallback()
primal_heur_root = PrimalHeurCallback(only_root=True)

models = {
    "Subgrad": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt
        },
    "SubgradPrimal": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur]
        },
    "SubgradPrimalRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, 
                      col_callback, opt_red_callback]
        },
    "SubgradPrimalRootRed": {
        "branch_strategy": branch_reduced_costs, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur_root, lagr_callback, 
                      col_callback, opt_red_callback]
        },
    "SubgradPrimalRedBeasleyBranch": {
        "branch_strategy": branch_beasley, 
        "lb_strategy": subgrad_opt,
        "callbacks": [primal_heur, lagr_callback, 
                      col_callback, opt_red_callback]
        },
    "LPPrimalCost": {
        "branch_strategy": branch_cost, 
        "lb_strategy": lp_rel,
        "callbacks": [primal_heur, lagr_callback, 
                      col_callback, opt_red_callback]},
    "LPPrimalBeasley": {
        "branch_strategy": branch_beasley, 
        "lb_strategy": lp_rel,
        "callbacks": [primal_heur, lagr_callback, 
                      col_callback, opt_red_callback]},
    "DualRed": {
        "branch_strategy": branch_beasley, 
        "lb_strategy": dual_lb,
        "callbacks": [primal_heur, lagr_callback, 
                      col_callback, opt_red_callback]},
}
short_names = {
    "Gurobi": "Gurobi",
    "Subgrad": "S",
    "SubgradPrimal": "SP",
    "SubgradPrimalRed": "SPR",
    "SubgradPrimalRootRed": "SPRR",
    "SubgradPrimalRedBeasleyBranch": "SPRB",
    "LPPrimalCost": "LPC",
    "LPPrimalBeasley" : "LPB",
    "DualRed" : "DR"
}
col_names = ["S", "SP", "SPR", "SPRR", "SPRB", "LPC", "LPB", "DR"]

In [ ]:
data = {}
time_limit = 60 * 5
for rows, cols, density in tqdm(
    [(7, 15, 0.3), (7, 15, 0.5), (7, 15, 0.7), 
     (10, 20, 0.3), (10, 20, 0.5), (10, 20, 0.7),
     (13, 22, 0.3), (13, 22, 0.5), (13, 22, 0.7),
     (15, 25, 0.3), (15, 25, 0.5), (15, 25, 0.7)],
     desc="Problem:", leave=False):
    res = {}
    A, b = generate_problem(rows, cols, density)
    
    start = time.process_time()
    m = gp.Model()
    m.Params.LogToConsole = 0
    x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
    m.setObjective(np.sum(A, axis=0) @ x)
    m.addConstr(A @ x >= b)
    m.optimize()
    elapsed = time.process_time() - start
    opt_gurobi = m.getObjective().getValue()
    
    x = []
    for v in m.getVars():
        x.append(v.x)
    x = np.array(x)
    
    # Make sure that the solution is feasible
    assert np.all(A @ x >= b)

    res["Gurobi"] = (elapsed, m.NodeCount)

    for model, config in tqdm(
        models.items(), desc="Model:", leave=False):
        start = time.process_time()
        bb = BranchAndBound(**config, time_start=start, 
                            time_limit=time_limit)
        try:
            bb.search(A, b)
            elapsed = time.process_time() - start
            x = np.zeros(A.shape[-1])
            x[bb.best.x1] = 1
            
            # Make sure that the solution is feasible
            assert np.all(A @ x >= b)
            opt = np.sum(A, axis=0) @ x

            # Check that the solution is optimal
            assert opt == opt_gurobi
            res[model] = (elapsed, bb.node_count)
        except TimeLimitException:
            res[model] = (np.nan, np.nan)
        
    data[(rows, cols, density)] = res

In [ ]:
with open("results/models.json", "w") as f:
    nd = {str(k): v for k, v in data.items()}
    json.dump(nd, f)

In [ ]:
with open("results/models.json", "r") as f:
    results = json.load(f)

In [ ]:
time_dict = {
    tuple(k[1:-1].split(", ")): {
        short_names[model]: values[0] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(time_dict)
df = df.T.round(2)
df = df.rename_axis(["Rows", "Columns", "Density"])
df

In [ ]:
s = df.style.format("{:.2f}") \
            .highlight_min(subset=col_names, 
                           axis=1, props="bfseries: ;")
print(s.to_latex(column_format="lllcccccccccc", 
                 hrules=True, 
                 clines="skip-last;data",
                 caption="Runtime of different configurations of the branch-and-bound algorithm over randomly generated TRAP instances with different numbers of rows and columns and different densities. Configurations whose runtime exceeded 5 minutes were assigned nan values. The best result obtained by algorithms other than Gurobi is highlighted in bold for each problem instance. The runtime was measured in seconds.",
                 label="tab:results:runtime", 
                 position_float="centering"))

In [ ]:
nodes_dict = {
    tuple(k[1:-1].split(", ")): {
        short_names[model]: values[1] for model, values in v.items()}  
    for k, v in results.items()}
df = pd.DataFrame(nodes_dict)
df = df.T
df = df.rename_axis(["Rows", "Columns", "Density"])
df

In [ ]:
s = df.style.format("{:.0f}") \
            .highlight_min(subset=col_names, 
                           axis=1, props="bfseries: ;")
print(s.to_latex(column_format="lllcccccccccc", 
                 hrules=True, 
                 clines="skip-last;data",
                 caption="Number of nodes generated by different configurations of the branch-and-bound algorithm over randomly generated TRAP instances with different numbers of rows and columns and different densities. Configurations whose runtime exceeded 5 minutes were assigned nan values. The best result obtained by algorithms other than Gurobi is highlighted in bold for each problem instance.",
                 label="tab:results:nodes", 
                 position_float="centering"))